# Розрахункова графічна робота

In [4]:
import cv2
import numpy as np
from datetime import datetime

def main():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Помилка: Не вдалося відкрити камеру.")
        return

    face_cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
    face_cascade = cv2.CascadeClassifier(face_cascade_path)

    orb = cv2.ORB_create(nfeatures=500)

    current_filter = 'normal' 

    print(" [N] - Нормальний режим")
    print(" [G] - Чорно-білий")
    print(" [B] - Розмиття")
    print(" [C] - Контрастність")
    print(" [E] - Виявлення країв")
    print(" [W] - ESP")
    print(" [S] - Зберегти скріншот")
    print(" [Q] - ВИХІД")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        output_frame = frame.copy()

        if current_filter == 'gray':
            output_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            output_frame = cv2.cvtColor(output_frame, cv2.COLOR_GRAY2BGR)
            
        elif current_filter == 'blur':
            output_frame = cv2.GaussianBlur(frame, (15, 15), 0)
            
        elif current_filter == 'contrast':
            output_frame = cv2.convertScaleAbs(frame, alpha=1.5, beta=0)
            
        elif current_filter == 'edge':
            edges = cv2.Canny(frame, 100, 200)
            output_frame = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
            output_frame = cv2.bitwise_not(output_frame)

        elif current_filter == 'esp':
            gray_for_faces = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray_for_faces, scaleFactor=1.3, minNeighbors=5)
            
            for (x, y, w, h) in faces:
                cv2.rectangle(output_frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(output_frame, "TARGET", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
            
            output_frame = cv2.convertScaleAbs(output_frame, alpha=0.7, beta=-30)

        cv2.putText(output_frame, f" {current_filter.upper()}", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        cv2.imshow('RGR', output_frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'): break
        elif key == ord('n'): current_filter = 'normal'
        elif key == ord('g'): current_filter = 'gray'
        elif key == ord('b'): current_filter = 'blur'
        elif key == ord('c'): current_filter = 'contrast'
        elif key == ord('e'): current_filter = 'edge'
        elif key == ord('w'): current_filter = 'esp'
        elif key == ord('s'):
            filename = f"screenshot_{datetime.now().strftime('%H%M%S')}.png"
            cv2.imwrite(filename, output_frame)
            print(f"Скріншот збережено: {filename}")

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'cv2'